#### Using Tools in LlamaIndex

Defining a clear set of Tools is crucial to performance. As we discussed in unit 1, clear tool interfaces are easier for LLMs to use. Much like a software API interface for human engineers, they can get more out of the tool if it’s easy to understand how it works.

There are four main types of tools in LlamaIndex:

1. FunctionTool: Convert any Python function into a tool that an agent can use. It automatically figures out how the function works.
2. QueryEngineTool: A tool that lets agents use query engines. Since agents are built on query engines, they can also use other agents as tools.
Toolspecs: Sets of tools created by the community, which often include tools for specific services like Gmail.
3. Utility Tools: Special tools that help handle large amounts of data from other tools.

We will go over each of them in more detail below.


### Creating a FunctionTool

A FunctionTool provides a simple way to wrap any Python function and make it available to an agent. You can pass either a synchronous or asynchronous function to the tool, along with optional name and description parameters. The name and description are particularly important as they help the agent understand when and how to use the tool effectively. Let’s look at how to create a FunctionTool below and then call it.

In [1]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny"

tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location."
)

tool.call("New York")

Getting weather for New York


ToolOutput(blocks=[TextBlock(block_type='text', text='The weather in New York is sunny')], tool_name='my_weather_tool', raw_input={'args': ('New York',), 'kwargs': {}}, raw_output='The weather in New York is sunny', is_error=False)

### Creating a QueryEngineTool

The QueryEngine we defined in the previous unit can be easily transformed into a tool using the QueryEngineTool class. Let’s see how to create a QueryEngineTool from a QueryEngine in the example below.

I will use Ollama for my purpose

Use pip install llama-index-vector-stores-chroma to install chroma vector store

In [4]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

llm = Ollama(model="kimi-k2:1t-cloud", base_url='http://127.0.0.1:11434')
query_engine = index.as_query_engine(llm=llm)
tool = QueryEngineTool.from_defaults(query_engine, name="some useful name", description="some useful description")

### Creating Toolspecs
Think of ToolSpecs as collections of tools that work together harmoniously - like a well-organized professional toolkit. Just as a mechanic’s toolkit contains complementary tools that work together for vehicle repairs, a ToolSpec combines related tools for specific purposes. For example, an accounting agent’s ToolSpec might elegantly integrate spreadsheet capabilities, email functionality, and calculation tools to handle financial tasks with precision and efficiency.

Install the Google Toolspec
As introduced in the section on the LlamaHub, we can install the Google toolspec with the following command:

```python
pip install llama-index-tools-google
```

And now we can load the toolspec and convert it to a list of tools.

In [6]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()

To get a more detailed view of the tools, we can take a look at the metadata of each tool.

In [7]:
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\n\nSearches email messages given a query string and the maximum number\nof results requested by the user\n   Returns: List of relevant message objects up to the maximum number of results.\n\nArgs:\n    query (str): The user's query\n    max_results (Optional[int]): The maximum number of search results\n    to return."),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\n\nCreate and insert a draft email.\n   Print the returned draft's message and id.\n   Returns: Draft object, including draft id and message meta data.\n\nArgs:\n    to (Optional[str]): The email addresses to send the message to\n    subject (Optional[str]): The subject for the event\n    message (Optional[str]): The message for the event"),

### Model Context Protocol (MCP) in LlamaIndex
LlamaIndex also allows using MCP tools through a ToolSpec on the LlamaHub. You can simply run an MCP server and start using it through the following implementation.

Installation command:
```python
pip install llama-index-tools-mcp
```

Below code is only for representation, it doesn't works as of now, I will update this code block after completing MCP course

In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)